# <font color='navy'> <b> Vytvorenie simulačného modelu pomocou programu *gmsh*</b></font> 

Úlohou predprocesora je vytvorenie simulačného modelu, prostredia a vygenerovanie vstupných dát pre simulačné prostredie. Vytvorený model sa často používa po simulácii v post-procesore pri zobrazení výsledkov, pretože výsledkom simulácie bývajú zvyčajne dáta o simulovanom jave (napr. intenzita elektrického poľa) a vlastný model nemusí byť súčasťou týchto dát.      

## <font color='teal'> <b> Vytvorenie modelu pomocou programu *gmsh* </b></font>

[Gmsh](http://gmsh.info/) je 2D/3D generátor siete modelu, je možno ho použiť aj na zobrazenie výsledkov simulácií. Je súčasťou komplexného prostredia [OneLab](http://onelab.info/), v ktorom je integrovaný univerzálny solver pre riešenie problémov. Aktuálne je podľa štatistík (Elmer) najpoužívanejším multifyzikálnym simulátorom v akademickom prostredí $(44\%)$.


Vytvorenie modelu pozostáva z definovania elementárnych entít - bodov, spojovacích línií, vytvorení plôch a objemov. Na základe elementárnych entít sú definované fyzické skupiny, z ktorých sú potom generované siete pre konečné prvky. Fyzické skupiny môžu pozostávať z viacerých elementárnych entít. Manuál ku gmsh - [link](https://bthierry.pages.math.cnrs.fr/tutorial/gmsh/).

Pre vytvorenie modelu môžeme použiť niekoľko postupov, prípadne ich vzájomne kombinovať.


### <font color='purple'> <b> Metóda 1. Vytvorenie modelu manuálne v GUI </b></font>
    
Manuálnym zadaním je vhodné vytvárať len veľmi jednoduché modely pre overenie funkčnosti prostredia (aj keď tvorcovia *gmsh* sú iného názoru), konfigurácií a v pedagogickom procese pri demonštrácii princípov pri vytváraní modelu. 

**Postup** 

* Vytvorenie sústavy bodov
    * *Modules - Geometry - Elementary entities - Add - Point*
* Vytvorenie prepojovacích línií - sú orientované (začiatok - koniec)  
    * *Modules - Geometry - Elementary entities - Add - Line*
* Vytvorenie plôch - 2D plocha
    * *Modules - Geometry - Elementary entities - Add - Plane surface*
    * Referenčným bodom plochy je jej stred
  
* Konverzia elementárnej plochy na fyzický komponent (môže pozostávať napr. z viacerých logických plôch a pod.)
    * *Modules - Geometry - Physical groups - Add - Surface*
  
* Vytvorenie siete
    * *Mesh - Define - 2D*
* Zjemnenie siete
    * *Mesh - Define - Refine by splitting*
  
Výsledok vytvárania modelu a siete sa automaticky zapisuje do súboru *\*.geo*, ktorý je možné editovať a použiť ako skript (nižšie).

**Príklad vygenerovanej siete**

<img src="./img/gmsh_01.png" width="600">


### <font color='purple'> <b> Metoda 2. Vytvorenie modelu skriptom </b></font>


*Gmsh* má vlastný skriptovací jazyk pre definovanie modelov, jednotlivé príkazy sa zadávajú do textového súboru, ktorý sa načíta pri spustení programu. Pri zadaní bodov zároveň definujeme aj hustotu generovanej siete v okolí bodu. 

**Príklad fragmentu skriptu zo súboru *[ex_01.geo](./data/ex_01.geo)***

    // elementarne entity
    Point(1) = {-10,  10, 0, 1.0};          // bod x,y,z,hustota siete v okoli bodu
    Point(4) = { 10,  10, 0, 1.0};
    Line(1) = {1, 4};                       // usecka z dvoch bodov
                                            // vytvorenie obrysu - uzatvorenej krivky
    Line Loop(5) = {3, 4, 1, 2, 13, 14, 11, 12, 23, 24, 21, 22}; 
    Plane Surface(6) = {5};                 // vytvorenie plochy z obrysu    
    ...
    
    // fyzicke entity
    Physical Line("e1") = {11, 14, 13, 12}; // lomena ciara zlozena z useciek
    ..
    Physical Surface("air") = {6};          // fyzicka plocha - moze byt zlozena z viacerych 
    
**Model vygenerovaný skriptom *ex_01.geo***

<img src="./img/gmsh_02.png" width="600">


### <font color='purple'> <b> Metóda 3. Vytvorenie parametrického modelu programom - skriptom </b></font>

Pre problémy spojené so zmenou parametrov modelu, napríklad pre potreby optimalizácie, je potrebné mať model alebo jeho časti zadané pomocou parametrov (napr. rozmery, poloha prvkov a pod.), ktoré môžeme meniť. Pre generovanie parametrického modelu existujú podporné programy a knižnice, jedným z nich je knižnica [pygmsh](https://pypi.org/project/pygmsh/).

**Inštalácia**

    sudo dnf install python3-gmsh
    pip install pygmsh
    
V *pygmsh* je možné model vytvoriť pomocou API, ktoré pozostáva z dvoch častí

* emulujúcich príkazy skriptu *gmsh* (add_point, add_line ...)
* funkcií kompatibilných s [openCASCADE](https://dev.opencascade.org/doc/overview/html/technical_overview.html) určených  pre vytváranie a modelovanie 2D a 3D objektov (add_ball, add_box ...) a operácií nad nimi (union, difference, extrude ...). Vygenerované modely je možné exportovať v rôznych formátoch pre použitie v post-procesingu, napríklad v programe *Paraview*.

In [2]:
# vytvorenie modelu pomocou gmsh skriptu
import pygmsh

with pygmsh.geo.Geometry() as geom:
    geom.add_polygon(
        [
            [0.0, 0.0],
            [1.0, -0.2],
            [1.1, 1.2],
            [0.1, 0.7],
        ],
        mesh_size=0.1,
    )
    # export geometrie modelu
    geom.save_geometry('./data/skript_gmsh_01.geo_unrolled')

    # generovanie mriezky - vtk format
    mesh = geom.generate_mesh()
    mesh.write("./data/skript_gmsh_01.vtk")


Info    : Writing './data/skript_gmsh_01.geo_unrolled'...
Info    : Done writing './data/skript_gmsh_01.geo_unrolled'


<img src="./img/scr_gmsh_01.png" width="600">

In [3]:
# vytvorenie modelu pomocou skriptu pre simulacne prostredie OpenCascade
import gmsh
import pygmsh

gmsh.initialize()
gmsh.option.setNumber("General.Terminal",0)

with pygmsh.occ.Geometry() as geom:
    geom.characteristic_length_min = 0.1
    geom.characteristic_length_max = 0.3

    rectangle = geom.add_rectangle([-1.0, -1.0, 0.0], 2.0, 2.0)
    disk1 = geom.add_disk([-1.2, 0.0, 0.0], 0.5)
    disk2 = geom.add_disk([+1.2, 0.0, 0.0], 0.5)

    disk3 = geom.add_disk([0.0, -0.9, 0.0], 0.5)
    disk4 = geom.add_disk([0.0, +0.9, 0.0], 0.5)

    flat = geom.boolean_union([rectangle, disk1, disk2, disk3, disk4]) 

    geom.extrude(flat, [0, 0, 0.3])

    # zapis geometrie modelu v roznych formatoch
    geom.save_geometry('./data/skript_gmsh_02.geo_unrolled')
    geom.save_geometry('./data/skript_gmsh_02.brep')
    geom.save_geometry('./data/skript_gmsh_02.step')

    mesh = geom.generate_mesh()
    mesh.write("./data/skript_gmsh_02.vtk")



*******************************************************************
******        Statistics on Transfer (Write)                 ******

*******************************************************************
******        Transfer Mode = 0  I.E.  As Is       ******
******        Transferring Shape, ShapeType = 0                      ******
** WorkSession : Sending all data
 Step File Name : ./data/skript_gmsh_02.step(1077 ents)  Write  Done


<br>
Zobrazenie vygenerovanej siete v *gmsh*, mriezka neobsahuje fyzicke entity.  Ak pri generovaní modelu vypíše skript hlásenie 

    Error   : Unknown number option 'General.NativeFileChooser'

treba zmazať konfiguračný súbor pre *gmsh* v hlavnom adresári **.gmshrc** 

<br>
<img src="./img/skript_gmsh_02.png" width="600">



<br>

V *gmsh* môžeme vytvoriť sieť a objekt exportovať do formátu [*VTK*](http://www.cacr.caltech.edu/~slombey/asci/vtk/vtk_formats.simple.html) (*Visualisation Toolkit*), ktorý vie načítať program pre post-processing [*ParaView*](https://www.paraview.org/).

<br>
<img src="./img/para_01.png" width="600">

### <font color='purple'> <b> Metóda 4. Tvorba modelu pomocou CAD programov </b></font>

Komplexnejšie modely je možné tvoriť pomocou open-source CAD programov, program FreeCAD poskytuje aj možnosť skriptovania modelu v Pythone. Typ modelu závisí od predpokladaného typu simulácie, napr. pre simuláciu elektrostatického poľa - kde nás zaujíma tvar elektrického poľa v prostredí - vytvárame model v tvare dutín v prostredí, kde jednotlivé plochy dutín reprezentujú elekródy so stanoveným potenciálom alebo okrajové podmienky.

<img src="./img/fc_01.png" width="600">

Vo FreeCad-e exportujeme model vo formáte BREP, ktorý vie čítať generátor mriežky *gmsh*. 

Súbor *\*.brep* obsahuje model rozložený na elementárne entity, neobsahuje ale fyzické entity. Mená objektov z FreeCAD-u nie sú súčasťou formátu *\*.brep* (a zrejme by to prinášalo problémy), preto musíme jednotlivé fyzické entity "poskladať" ručne - doplnením do súboru *\*.geo*.

Zásadným rozdielom medzi elementárnou a fyzickou entitou je tem, že jedna fyzická entita (napr. elektróda, zemná plocha atď.) môže byť poskladaná z viacerých fyzických entít. Naopak - jeden objekt v CAD-e (napr. valec) môže (ako sústava 3 entít - dno, dekel, plášť) reprezentovať samostatné fyzické entity (2 elektródy spojené izolátorom).

**Priradenie fyzických entít - Príklad *ex_03.geo***  

Do súboru importujeme vygenerovaný súbor *ex_03.brep* a doplníme definície fyzických entít. Nepríjemné je hlavne určenie jednotlivých plôch objektov, naštastie sú generované v jednotlivých objektoch v poradí a pomôcť si môžeme aj zobrazením označenia entít v *gmsh*. 
    
    Merge "exp_03.brep";
    Physical Volume("air") = {1};

    Physical Surface("s2")  = {7,8,9};
    Physical Surface("q1")  = {10,11,12};
    Physical Surface("ext") = {1,2,3,4,5,6};

<img src="./img/scr_gmsh_02.png" width="600">


## <font color='teal'> <b> Generovanie mriežky v programe *gmsh* </b></font>

V program *gmsh* môžeme vytvárať mriežku podľa potreby jej zjemnením, vyhladením a pod. Program (ver. 4.0 a vyššia) automaticky pri ukladaní zvolí formát mriežky vo verzii 4, ktorú simulačný engine ElmerCSC (zatiaľ) nepozná, preto je potrebné pri ukladaní zvoliť export mriežky vo formáte 2.0 (ASCII 2).

Vygenerovanú sieť môžeme skontrolovať v grafickom rozhraní ElmerGUI (s obmedzeniami ...).

<img src="./img/elmer_01.png" width="600">


### <font color='purple'> <b> Generovanie mrižky skriptom </b></font>

Pri častejších zmenách modelu je ručné opakované generovanie mriežky zdĺhavé, môžeme si pomôcť vygenerovaním siete priamo v súbore *\*.geo* 

**Konfigurácia automatického generovania mriežky - Príklad *ex_04.geo***

    Merge "ex_03.brep";
    Physical Volume("air") = {1};

    Physical Surface("s2")  = {7,8,9};
    Physical Surface("q1")  = {10,11,12};
    Physical Surface("ext") = {1,2,3,4,5,6};

    Mesh 3;
    RefineMesh;
    RefineMesh;
    
Sieť vygenerujeme a zapíšeme v správnom formáte v povelovom riadku príkazom

     gmsh ex_04.geo -3 -format msh2 -save_all -o ex_04.msh 

--------------